In [ ]:
import os
import numpy as np
import gzip
import pickle

import import_ipynb
from constants import *
from simulation import simul
from utils import *

In [ ]:
def save_mask(save_dir, file_num, mask, seed_size, infl_orig, infl_masked, time_alg):
    with open(save_dir+f'/mask_{file_num}.txt','w') as f:
        f.write('seed infl_orig infl_masked reldec%\n')
        f.write(f'{seed_size} {infl_orig} {infl_masked} {(infl_orig-infl_masked)/(infl_orig-seed_size)}\n')
        f.write('u v p time%\n')
        for i in range(len(mask)):
            u,v,p = mask[i]
            t = time_alg[i]
            f.write(f'{int(u)} {int(v)} {p} {t}\n')


def save_record(save_dir, record, time_pretrain):
    
    avgtime = np.mean(record[:,3])
    avgreldec = np.mean((record[:,1]-record[:,2])/(record[:,1]-record[:,0]))
    with open(save_dir+'/summary.txt','w') as f:
        f.write(f'avgreldec%: {avgreldec}, avgtime: {avgtime}\n')
        f.write(f'pretrain time: {time_pretrain}\n')
        f.write('seed infl_orig infl_masked reldec% time\n')
        for seed_size, infl_orig, infl_masked, time_alg in record: f.write(f'{seed_size} {infl_orig} {infl_masked} {(infl_orig-infl_masked)/(infl_orig-seed_size)} {time_alg}\n')
            
            
def make_subresult(alg_name, dataset_name, from_del_edge_num, to_del_edge_num, save_tag=None):
    
    dataset_tag = {'wiki_indeg-50.pkl.gz':'wiki', 'Extended_test_LP-50.pkl.gz':'E', 'Celebrity_test_LP-50.pkl.gz':'C', 'WannaCry_test_LP-50.pkl.gz':'W'}[dataset_name]
    from_save_dir = RESULT_DIR+dataset_tag+'/'+str(from_del_edge_num)+'/'+alg_name
    to_save_dir = RESULT_DIR+dataset_tag+'/'+str(to_del_edge_num)+'/'+alg_name
    if save_tag : from_save_dir += save_tag; to_save_dir += save_tag
    if not os.path.exists(from_save_dir): print('no such directory'); return
    if not os.path.exists(to_save_dir): os.makedirs(to_save_dir)
    
    
    graph_name = dataset_name.split('-')[0]
    n,m,adj_list = txt2adj(graph_name)
    with gzip.open(DATASET_DIR+dataset_name, 'rb') as f: rawdata = pickle.load(f)

    record = []
    for file_num, (is_seed, probs) in enumerate(rawdata): 
        mask = []
        time_alg = []
        with open(from_save_dir+f'/mask_{file_num}.txt','r') as f:
            f.readline()
            seed_size, infl_orig, _, _ = map(float,f.readline().split())
            f.readline()
            for i in range(to_del_edge_num):
                u,v,p,t = map(float,f.readline().split())
                mask.append((int(u),int(v),p))
                time_alg.append(t)
        seed_idx = np.where(is_seed==1)[0]

        
        adj_mat_masked = list2mat(adj_list)
        for u,v,p in mask: adj_mat_masked[u][v]=0
        adj_list_masked = mat2list(adj_mat_masked)

        
        infl_masked = sum(simul(adj_list_masked,seed_idx))
        
        record.append([len(seed_idx),infl_orig,infl_masked,time_alg[-1]])

        
        save_mask(to_save_dir, file_num, mask, len(seed_idx), infl_orig, infl_masked, time_alg)

    record = np.array(record)
    avgtime = np.mean(record[:,3])
    avgreldec = np.mean((record[:,1]-record[:,2])/(record[:,1]-record[:,0]))
    

    with open(from_save_dir+'/summary.txt','r') as f:
        f.readline()
        time_pretrain = float(f.readline().split()[-1])
    
    
    save_record(to_save_dir, record, time_pretrain)
    return avgreldec, record, time_pretrain